In [1]:
import quandl
from sklearn.preprocessing.data import StandardScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, BatchNormalization, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential

quandl.ApiConfig.api_key = 'MdTYhiXQR_LnUHzq-qma'

# State the Real Life Model
companies = ['WIKI/AAPL', 'WIKI/JNJ', 'WIKI/PG', 'WIKI/JPM', 'WIKI/KO', 'WIKI/XOM', 'WIKI/MCD']  # The companies
obj = ['Close']  # The Data AI will get to see
prev_days = 16  # How many days back the AI get to see

fields = []
for company in companies:
    for o in obj:
        fields.append(company + ' - ' + o)

x = quandl.get(companies)[fields]
x.dropna(inplace=True)

# Percent Change Data
chg_x = x.pct_change()
chg_x.dropna(inplace=True)

# norm object Normalizes the data
# 'data' and 'norm' objects contains all information about the original data
#norm = StandardScaler()
#norm.fit(chg_x)
#data = norm.transform(chg_x)
data = chg_x.to_numpy()

x_all = []
y_all = []

# appending 0.0 at the end to represent holding the money will result in 0.0 percent change
for i in range(len(data) - prev_days - 1):
    x_all.append(data[i:i + prev_days])
    y_all.append(np.append(data[prev_days + i], 0.0))

x_all = np.array(x_all)
y_all = np.array(y_all)
x_train = x_all[:7500]
y_train = y_all[:7500]
x_test = x_all[7500:]
y_test = y_all[7500:]

/home/hina/anaconda3/envs/alexV2/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.preprocessing.data module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
def neg_profit(y_true, y_pred):
    multiplier = tf.constant(np.append(norm.scale_, 0.0), dtype=tf.float32)
    adder = tf.constant(np.append(norm.mean_, 0.0), dtype=tf.float32)
    return tf.tensordot(tf.add(tf.multiply(y_true, multiplier), adder), tf.cast(y_pred, tf.float32), 0) * -100

def neg_prof(y_true, y_pred):
    return tf.tensordot(y_true, y_pred, 0) * -100

In [6]:
model = Sequential()
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(LSTM(128))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(8, activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.000001), loss=neg_prof)
model.fit(x_train, y_train, batch_size=1024, epochs=5, validation_data=(x_test, y_test))

Epoch 1/5
8/8 [==============================] - 1s 105ms/step - loss: -0.0042 - val_loss: -0.0027
Epoch 2/5
8/8 [==============================] - 0s 39ms/step - loss: -0.0042 - val_loss: -0.0027
Epoch 3/5
8/8 [==============================] - 0s 39ms/step - loss: -0.0042 - val_loss: -0.0027
Epoch 4/5
8/8 [==============================] - 0s 39ms/step - loss: -0.0042 - val_loss: -0.0027
Epoch 5/5
8/8 [==============================] - 0s 39ms/step - loss: -0.0042 - val_loss: -0.0027


In [ ]:
z = model.predict(x_test)

In [ ]:
test_changes = norm.inverse_transform(y_test[:, 0:7])

In [ ]:
z = z[:, 0:7]

In [ ]:
z.shape

In [ ]:
test_changes.shape

In [ ]:
w = []
for i in range(1112):
    w.append(np.dot(z[i], test_changes[i]))

In [ ]:
w

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(50), w[400:450])